In [1]:
import numpy as np
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, pick_types, events_from_annotations
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM

# Download EEG dataset (You might need to adjust the subject and runs based on your dataset)
subject = 1
runs = [6, 10, 14]  # Motor imagery: hands vs feet
raw_fnames = eegbci.load_data(subject, runs)

# Concatenate runs
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])

# Extract events
events, _ = events_from_annotations(raw)

# Define event IDs
event_id = dict(hands=2, feet=3)

# Define EEG channels
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

# Define epochs
tmin, tmax = -1, 4  # Define epoch length
epochs = Epochs(raw, events, event_id, tmin, tmax, proj=False, picks=picks, baseline=None, preload=True)

# Prepare features and labels
X = epochs.get_data()  # EEG data
y = epochs.events[:, -1]  # Event labels

# Feature extraction: covariance matrices
covs = Covariances().fit_transform(X)

# Classification pipeline
clf = make_pipeline(Covariances(), TangentSpace(), MDM())

# Train-test split (You can use more sophisticated cross-validation methods)
train_size = 0.8
train_samples = int(train_size * len(X))
X_train, X_test = covs[:train_samples], covs[train_samples:]
y_train, y_test = y[:train_samples], y[train_samples:]

# Train the classifier
clf.fit(X_train, y_train)

# Evaluate the classifier
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)


Download complete in 37s (7.4 MB)
Extracting EDF parameters from C:\Users\ABC\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\ABC\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\ABC\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
45 matching events found
No baseline correction applied
Using data from preloaded Raw for 45 events and 801 original time points ...
0 bad epochs dr

C:\Users\ABC\AppData\Local\Temp\ipykernel_17752\1152810384.py:33: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()  # EEG data
c:\Users\ABC\anaconda3\envs\py38\lib\site-packages\pyriemann\utils\base.py:18: RuntimeWarning: invalid value encountered in log
  eigvals = operator(eigvals)


ValueError: Matrices must be positive definite. Add regularization to avoid this error.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from pyriemann.utils.mean import mean_covariance
from pyriemann.utils.distance import distance_riemann

# Assume covs_train and covs_test are the covariance matrices for training and testing data
# Assume labels_train and labels_test are the corresponding labels

class MDMClassifier:
    def __init__(self):
        self.class_means = None

    def fit(self, X_train, y_train):
        unique_classes = np.unique(y_train)
        self.class_means = {}
        for cls in unique_classes:
            class_covs = X_train[y_train == cls]
            class_mean = mean_covariance(class_covs, metric='riemann')
            self.class_means[cls] = class_mean

    def predict(self, X_test):
        y_pred = []
        for sample in X_test:
            min_dist = float('inf')
            predicted_class = None
            for cls, class_mean in self.class_means.items():
                dist = distance_riemann(sample, class_mean)
                if dist < min_dist:
                    min_dist = dist
                    predicted_class = cls
            y_pred.append(predicted_class)
        return np.array(y_pred)

# Assuming covs_train and covs_test are already computed covariance matrices
# Also assuming labels_train and labels_test are the corresponding labels

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(covs, labels, test_size=0.2, random_state=42)

# Instantiate and train the classifier
mdm_classifier = MDMClassifier()
mdm_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = mdm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
